In [ ]:
!pip3 install transformers

     |████████████████████████████████| 2.8 MB 6.9 MB/s 
     |████████████████████████████████| 636 kB 59.7 MB/s 
     |████████████████████████████████| 3.3 MB 42.8 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
     |████████████████████████████████| 895 kB 75.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, BertModel,AutoModel
import pandas as pd
import torch
import numpy as np
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import os
from transformers import TextClassificationPipeline

In [ ]:
def bert_predict(model, tokenizer, text):
  prediction_prob = []
  classification_pipeline = TextClassificationPipeline(model=model,
                                                                tokenizer=tokenizer,
                                                                framework='pt',
                                                                binary_output=True,
                                                                return_all_scores=True)
  for i in text:
      positive_label_probability = classification_pipeline(i)[0][0]["score"]
      prediction_prob.append(positive_label_probability)
  return prediction_prob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print('We will use the GPU:', torch.cuda.get_device_name(0))

We will use the GPU: Tesla K80


In [ ]:
ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/Allen_NLP_hackathon'
NYC_DATA = os.path.join(ROOT_PATH, 'Data', 'sentences.csv')
BERT_MODELS = os.path.join(ROOT_PATH, 'Models')

In [ ]:
df = pd.read_csv(NYC_DATA, header=None).sample(1000)

In [ ]:
df['sentence1'] = df[0]
df.drop(0, axis=1, inplace=True)

In [ ]:
df

,sentence1
196475,"So for about six months , they trudged the bea..."
656375,"When the mass movements became influential , h..."
295809,"WITH : Peter Krause -LRB- Nate -RRB- , Michael..."
58031,"The prices range from 900 euros -LRB- $ 1,161 ..."
555198,"In 1990 , after Indianapolis took Illinois qua..."
...,...
373872,Lewis said the others were James C. Lessersohn...
210479,"Across Europe , several countries plan to hono..."
546142,"Olevia 's competitors include Vizio , which ha..."
21807,New pop CD 's to be released tomorrow include ...


In [ ]:
# load model
model = BertForSequenceClassification.from_pretrained(BERT_MODELS+"/anger/")
# load tokenizer
tokenizer = BertTokenizer.from_pretrained(BERT_MODELS+"/anger/")

In [ ]:
anger_predicitons = bert_predict(model, tokenizer, df.sentence1.values)

In [ ]:
df["anger_prediciton"] = anger_predicitons

In [ ]:
df.to_csv(os.path.join(ROOT_PATH, 'Data', 'predicted_sentences.csv'))